# Geração do Conjunto de dados

In [ ]:
import requests
import pandas as pd
from config import SPOT_CLIENT_SECRET, SPOT_CLIENT_ID

# Função para obter o token de acesso ao Spotify
def get_spotify_token(client_id, client_secret):
    """
    Obtém um token de acesso ao Spotify.

    Parâmetros:
    client_id (str): ID do cliente Spotify
    client_secret (str): Segredo do cliente Spotify

    Retorna:
    str: Token de acesso
    """
    auth_url = "https://accounts.spotify.com/api/token"
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    headers = {"Authorization": f"Basic {auth_header}"}
    data = {"grant_type": "client_credentials"}

    response = requests.post(auth_url, headers=headers, data=data)
    if response.status_code != 200:
        raise Exception(f"Erro ao obter token. Código {response.status_code}: {response.text}")
    
    return response.json()['access_token']

# Obter o token de acesso
CLIENT_ID = SPOT_CLIENT_ID
CLIENT_SECRET = SPOT_CLIENT_SECRET
TOKEN = get_spotify_token(CLIENT_ID, CLIENT_SECRET)

# Cabeçalhos para solicitações à API do Spotify
headers = {"Authorization": f"Bearer {TOKEN}"}

# Função para buscar características de áudio de uma faixa específica
def get_audio_features(track_name, artist_name):
    """
    Busca características de áudio de uma faixa específica usando a API do Spotify.

    Parâmetros:
    track_name (str): Nome da faixa
    artist_name (str): Nome do artista

    Retorna:
    dict: Características de áudio da faixa ou None se não encontrada
    """
    response = requests.get(f"{BASE_URL}search?q=track:{track_name}+artist:{artist_name}&type=track", headers=headers)
    response_json = response.json()
    
    if 'tracks' not in response_json:
        print(f"Erro na resposta para: {track_name} de {artist_name}. Resposta: {response_json}")
        not_found_tracks.append([track_name, artist_name])
        return None

    items = response_json['tracks']['items']
    if not items:
        print(f"Não foi possível encontrar: {track_name} de {artist_name}")
        not_found_tracks.append([track_name, artist_name])
        return None
    
    track_id = items[0]['id']
    track_response = requests.get(f"{BASE_URL}audio-features/{track_id}", headers=headers)
    return track_response.json()

# Carregar dados do arquivo CSV
basedir = r"C:\Users\luboh\OneDrive\Área de Trabalho\USP\EngDadosBigData\eEDB-018_REPO\DEVELOPER\eEDB-018\prj\DATA"
data_file = basedir + r'\panda_dataset_taffc_metadata.csv'
data_source = pd.read_csv(data_file)

# Processamento de faixas
tracks = data_source[['Title','Artist','GenresStr','Quadrant']].values
not_found_tracks = []
data = []

for track_name, artist_name, genre, quadrant in tracks:
    audio_features = get_audio_features(track_name, artist_name)

    if audio_features is None:
        continue

    data.append([
        track_name, artist_name, genre, quadrant,
        audio_features['acousticness'], audio_features['danceability'],
        audio_features['energy'], audio_features['instrumentalness'],
        audio_features['key'], audio_features['liveness'],
        audio_features['loudness'], audio_features['mode'],
        audio_features['speechiness'], audio_features['tempo'],
        audio_features['time_signature'], audio_features['valence']
    ])

# Criação dos DataFrames
df = pd.DataFrame(data, columns=[
    'Nome da Faixa', 'Cantor ou Compositor', 'Genero', 'Quadrante',
    'acousticness', 'danceability', 'energy', 'instrumentalness',
    'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
    'time_signature', 'valence'
])
df_not_found = pd.DataFrame(not_found_tracks, columns=['Nome da Faixa', 'Cantor ou Compositor'])

# Salvando os DataFrames
df.to_csv('spotify_dataset.csv', index=False)
df_not_found.to_csv('not_found_spotify.csv', index=False)
